In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.utils.data.dataset import random_split
from torch.utils.data import DataLoader, TensorDataset
import torch


In [2]:
loaded_activations  = torch.load('activations.pth')
loaded_embeddings = torch.load("embeddings.pth")
loaded_residual_stream = torch.load("residual_stream.pth")

print(loaded_residual_stream[0].shape)

torch.Size([65, 20, 4096])


In [3]:
activations = [i[2][3, -1, :] for i in loaded_activations]
residual_stream =  [ torch.stack([i[n, -1, :] for i in loaded_residual_stream]) for n in range(0, 10)]

activations = torch.stack(activations)
#residual_stream = torch.stack(residual_stream)

training =  torch.cat( (torch.cat([residual_stream[i] for i in [3] ], 1), activations), 1 )


print(loaded_embeddings.shape, loaded_embeddings.dtype)
print(training.shape[1])

# Create a complete dataset
full_dataset = TensorDataset(training.to("cuda"), loaded_embeddings.to("cuda"))

# Define the sizes for your training and validation sets
total_size = len(full_dataset)
train_size = int(0.8 * total_size)
val_size = total_size - train_size

# Split the dataset
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Create DataLoaders for both training and validation sets
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32)  # No need to shuffle the validation data


torch.Size([195, 4096]) torch.float16
8192


In [5]:
import torch.nn as nn
import torch.nn.functional as F

class ImprovedTwoLayerNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ImprovedTwoLayerNN, self).__init__()
        # Increase depth and capacity
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.relu1 = nn.LeakyReLU()
        self.batch_norm1 = nn.BatchNorm1d(hidden_size)
        self.dropout1 = nn.Dropout(0.5)  # Adjust dropout rate as needed
        
        # Additional layer
        self.layer2 = nn.Linear(hidden_size, hidden_size * 2)
        self.relu2 = nn.LeakyReLU()
        self.batch_norm2 = nn.BatchNorm1d(hidden_size * 2)
        self.dropout2 = nn.Dropout(0.5)  # Adjust dropout rate as needed
        
        # Output layer
        self.layer3 = nn.Linear(hidden_size * 2, output_size)
    
    def forward(self, x):
        x = self.layer1(x)
        x = self.relu1(x)
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        
        x = self.layer2(x)
        x = self.relu2(x)
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        
        x = self.layer3(x)
        return x
    
class TwoLayerNN(nn.Module):
    def __init__(self, input_size, hidden_size,  output_size):
        super(TwoLayerNN, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        return x



In [9]:

# Hyperparameters
input_size = training.shape[1]
hidden_size = 10000
learning_rate = 0.001
num_epochs = 40


# Initialize model, loss, and optimizer

model = TwoLayerNN(input_size, hidden_size, 4096)
#model = ImprovedTwoLayerNN(input_size, hidden_size, 4096)

model.to("cuda")
criterion = nn.MSELoss()  
optimizer = optim.Adam(model.parameters(), lr=learning_rate)



for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_dataloader:
        inputs, targets = inputs.to("cuda").float(), targets.to("cuda").float()
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
    
    epoch_loss = running_loss / len(train_dataloader.dataset)
    # Validation phase
    model.eval()  # Set the model to evaluation mode
    running_loss = 0.0
    with torch.no_grad():  # No gradients need to be calculated
        for inputs, targets in val_dataloader:
            inputs, targets = inputs.to("cuda").float(), targets.to("cuda").float()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            running_loss += loss.item() * inputs.size(0)
    val_loss = running_loss / len(val_dataloader.dataset)

    print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {epoch_loss:.4f}, Validation Loss: {val_loss:.4f}')




Epoch [1/40], Training Loss: 22.5960, Validation Loss: 21.2086
Epoch [2/40], Training Loss: 19.9887, Validation Loss: 17.0394
Epoch [3/40], Training Loss: 15.7569, Validation Loss: 13.9575
Epoch [4/40], Training Loss: 13.5084, Validation Loss: 13.3977
Epoch [5/40], Training Loss: 12.8835, Validation Loss: 12.3818
Epoch [6/40], Training Loss: 11.8563, Validation Loss: 11.8012
Epoch [7/40], Training Loss: 11.4453, Validation Loss: 11.7221
Epoch [8/40], Training Loss: 11.3480, Validation Loss: 11.4457
Epoch [9/40], Training Loss: 11.0023, Validation Loss: 11.0267
Epoch [10/40], Training Loss: 10.6200, Validation Loss: 10.8318
Epoch [11/40], Training Loss: 10.4886, Validation Loss: 10.7280
Epoch [12/40], Training Loss: 10.3536, Validation Loss: 10.5536
Epoch [13/40], Training Loss: 10.1843, Validation Loss: 10.4601
Epoch [14/40], Training Loss: 10.0995, Validation Loss: 10.3859
Epoch [15/40], Training Loss: 10.0171, Validation Loss: 10.2940
Epoch [16/40], Training Loss: 9.9262, Validation 

In [10]:
# Ensure the model is in evaluation mode
model.eval()



predictions = []
with torch.no_grad():  # No gradients needed for inference
    for inputs, _ in full_dataset:  # Assuming your dataset returns inputs and targets
        inputs = inputs.to('cuda').float().unsqueeze(0)
        
        # Get the model output
        outputs = model(inputs)
        

        predictions.append(outputs.cpu())

# Concatenate all batches of predictions
all_predictions = torch.stack(predictions, dim=0)

# Save the tensor to a file
torch.save(all_predictions, 'model_predictions.pth')

In [11]:
predictions

[tensor([[ 1.1851,  1.9739,  1.0377,  ...,  7.0628, -2.4395,  3.3619]]),
 tensor([[ 0.9378,  2.0632,  1.0800,  ...,  7.1391, -3.1234,  2.7384]]),
 tensor([[ 1.1774,  2.0113,  1.1116,  ...,  6.8351, -3.0224,  2.9552]]),
 tensor([[ 1.3200,  1.9634,  0.8198,  ...,  7.2429, -2.2022,  3.5344]]),
 tensor([[ 1.7550,  1.5351,  1.0508,  ...,  6.6703, -2.7565,  3.5197]]),
 tensor([[ 1.1092,  1.8253,  0.7763,  ...,  6.5863, -2.6959,  2.7108]]),
 tensor([[ 1.2174,  1.9298,  0.7212,  ...,  7.0552, -2.4219,  3.1423]]),
 tensor([[ 0.9978,  1.9279,  1.0411,  ...,  6.2475, -3.1752,  2.4179]]),
 tensor([[ 1.0171,  1.8925,  0.8713,  ...,  6.4512, -2.7539,  2.5472]]),
 tensor([[ 1.1101,  2.0591,  0.7049,  ...,  7.0183, -2.8054,  2.6537]]),
 tensor([[ 1.1161,  2.0581,  0.6805,  ...,  7.2390, -3.4533,  2.3615]]),
 tensor([[ 0.8318,  1.8835,  1.2477,  ...,  6.3916, -3.1897,  2.3691]]),
 tensor([[ 1.0533,  1.9651,  0.9165,  ...,  6.8442, -2.7186,  2.7821]]),
 tensor([[ 0.9381,  1.9861,  0.8526,  ...,  6.6828,